In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import torch
import torch.nn as nn
# import torch.nn.functional as F
from torch import optim
from torchinfo import summary
import torchsummary

from torchvision import transforms
from video_dataset import  VideoFrameDataset, ImglistToTensor
from pytorchtools import EarlyStopping
from tqdm import tqdm
import random
import numpy as np

from thop import profile
print(torch.__version__)

In [ ]:
manualSeed = 996
print("Random Seed: ", manualSeed)

if torch.cuda.is_available():
    device = torch.device('cuda')
#     torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')
    
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)
np.random.seed(manualSeed)
torch.backends.cudnn.deterministic = True

print(device)

In [ ]:
batch_size = 4
num_segments = 8
advprop = True
image_size = 224 #b0
# image_size = 299 #b3

if advprop:  # for models using advprop pretrained weights
    preprocess = transforms.Compose([
            ImglistToTensor(),  # list of PIL images to (FRAMES x CHANNELS x HEIGHT x WIDTH) tensor
            transforms.Resize(image_size),  # image batch, resize smaller edge to 299
            transforms.CenterCrop(image_size),  # image batch, center crop to square 299x299
            transforms.Lambda(lambda img: img * 2.0 - 1.0),
    ])
else:
    preprocess = transforms.Compose([
            ImglistToTensor(),  # list of PIL images to (FRAMES x CHANNELS x HEIGHT x WIDTH) tensor
            transforms.Resize(image_size),  # image batch, resize smaller edge to 299
            transforms.CenterCrop(image_size),  # image batch, center crop to square 299x299
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
videos_root = "../../UCF101_IMG/"

train1_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "train01"),
    annotationfile_path=os.path.join(videos_root, "train01/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=True,
    test_mode=False
)
train2_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "train02"),
    annotationfile_path=os.path.join(videos_root, "train02/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=True,
    test_mode=False
)
train3_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "train03"),
    annotationfile_path=os.path.join(videos_root, "train03/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=True,
    test_mode=False
)

test1_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "test01"),
    annotationfile_path=os.path.join(videos_root, "test01/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=False,
    test_mode=True
)
test2_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "test02"),
    annotationfile_path=os.path.join(videos_root, "test02/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=False,
    test_mode=True
)
test3_dataset = VideoFrameDataset(
    root_path=os.path.join(videos_root, "test03"),
    annotationfile_path=os.path.join(videos_root, "test03/annotations.txt"),
    num_segments=num_segments,
    frames_per_segment=1,
    imagefile_template='frame_{0:012d}.jpg',
    transform=preprocess,
    random_shift=False,
    test_mode=True
)

train1_dataloader = torch.utils.data.DataLoader(
    dataset=train1_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=batch_size,
    pin_memory=True
)
train2_dataloader = torch.utils.data.DataLoader(
    dataset=train2_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=batch_size,
    #pin_memory=True
)
train3_dataloader = torch.utils.data.DataLoader(
    dataset=train3_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=batch_size,
    #pin_memory=True
)

test1_dataloader = torch.utils.data.DataLoader(
    dataset=test1_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=batch_size,
    pin_memory=True
)
test2_dataloader = torch.utils.data.DataLoader(
    dataset=test2_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=batch_size,
    #pin_memory=True
)
test3_dataloader = torch.utils.data.DataLoader(
    dataset=test3_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=batch_size,
    #pin_memory=True
)

plot_test1_dataloader = torch.utils.data.DataLoader(
    dataset=test1_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
#     pin_memory=True
)
plot_test2_dataloader = torch.utils.data.DataLoader(
    dataset=test2_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    #pin_memory=True
)
plot_test3_dataloader = torch.utils.data.DataLoader(
    dataset=test3_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    #pin_memory=True
)

In [ ]:
epochs = 200
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-3
momentum = 0.9
weight_decay = 5e-4
milestones = [15,30,45]

gamma = 0.3

train1_size = len(train1_dataset)
test1_size = len(test1_dataset)
train2_size = len(train2_dataset)
test2_size = len(test2_dataset)
train3_size = len(train3_dataset)
test3_size = len(test3_dataset)

print(len(train1_dataset))
print(len(test1_dataset))

In [ ]:
def train(model, epochs, criterion, train_loader, test_loader, opt, scheduler, train_size, test_size, device, model_name):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    
    batch_cnt = 0
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=15, verbose=True)
#     early_stopping_cnt = 0
    max_acc = 0
#     model=nn.DataParallel(model,device_ids=[0, 1])
    model.to(device)
#     scaler = torch.cuda.amp.GradScaler()

    for epoch in range(1, epochs+1):
        correct = 0
        temp = 0
        acc = 0
        model.train()
        
        for data, label in tqdm(train_loader):
#             with torch.cuda.amp.autocast():

                batch_cnt = batch_cnt + 1

                x = data.to(device, dtype=torch.float)
                y = label.to(device, dtype=torch.long)

                pred = model(x)
                loss = criterion(pred, y)
                temp += loss.item()
                correct += (torch.max(pred, 1)[1] == y).sum().item()

                loss.backward()
#                 scaler.scale(loss).backward()

#                 if batch_cnt%2 == 0:
#                     scaler.step(opt)
#                     scaler.update()
#                     opt.zero_grad()
                opt.step()
                opt.zero_grad()
        
        scheduler.step()
        
        acc = correct / train_size
        train_loss.append(temp/len(train_loader))
        train_acc.append(acc)
        
        correct = 0
        temp = 0
        acc = 0
    
        model.eval()
        
        with torch.no_grad():
            for data, label in tqdm(test_loader):
                
                #x = data.to(device, dtype=torch.float)
                x = data.cuda()
                x = x.float()
                y = label.to(device, dtype=torch.long)
            
                pred = model(x)
                loss = criterion(pred, y)
                temp += loss.item()
                correct += (torch.max(pred, 1)[1] == y).sum().item()
        
        acc = correct / test_size
        los = temp/len(test_loader)
        
        test_loss.append(los)
        test_acc.append(acc)
        
        print(f'epoch:{epoch}/{epochs} ', end='')
        if acc > max_acc:
            max_acc = acc
            torch.save(model, "./saved_model/"+model_name)
            early_stopping.counter = 0
            print(f"Saving model with better accuracy: {acc:.2%}", end='\n')
        else:
            print(f"Accuracy is not better than: {max_acc:.2%}", end='\n')
        
        early_stopping(test_loss[epoch-1], model)
        
        print(f"{opt.param_groups[0]['lr']}")
        print(f"train loss:{train_loss[epoch-1]:.2f}  train acc:{train_acc[epoch-1]:.2%}", end ='\n')
        print(f"test loss:{test_loss[epoch-1]:.2f}  test acc:{test_acc[epoch-1]:.2%}")
                
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    return train_acc, test_acc

# Shift Conv & DCT

## Split 1

In [ ]:
modelName = "EFN_Cle112_allGC_Final_Shift"

In [ ]:
from EfficientNet_GCShift import EfficientNet_GCShift

EFN_model_1 = EfficientNet_GCShift(device, num_segments, batch_size).to(device)
opt1 = optim.SGD(EFN_model_1.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

scheduler1 = optim.lr_scheduler.MultiStepLR(opt1, milestones=milestones,gamma=gamma)
# scheduler1 = optim.lr_scheduler.ReduceLROnPlateau(opt1, mode='min', factor=0.3, patience=3)
train1_acc = []
test1_acc = []

torchsummary.summary(EFN_model_1, (num_segments, 3, image_size, image_size))
summary(EFN_model_1, input_size=(4, num_segments, 3, image_size, image_size))

In [ ]:
train1_acc, test1_acc = train(EFN_model_1, epochs, criterion, train1_dataloader, test1_dataloader, opt1, scheduler1, train1_size, test1_size, device, modelName+"_1.pt")

## Split 2

In [ ]:
from EfficientNet_GCShift import EfficientNet_GCShift

EFN_model_2 = EfficientNet_GCShift(device, num_segments, batch_size).to(device)
opt2 = optim.SGD(EFN_model_2.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

scheduler2 = optim.lr_scheduler.MultiStepLR(opt2, milestones=milestones, gamma=gamma)
# scheduler1 = optim.lr_scheduler.ReduceLROnPlateau(opt1, mode='min', factor=0.3, patience=3)
train2_acc = []
test2_acc = []

torchsummary.summary(EFN_model_2, (num_segments, 3, image_size, image_size))

In [ ]:
train2_acc, test2_acc = train(EFN_model_2, epochs, criterion, train2_dataloader, test2_dataloader, opt2, scheduler2, train2_size, test2_size, device, modelName+"_2.pt")

## Split 3

In [ ]:
from EfficientNet_GCShift import EfficientNet_GCShift

EFN_model_3 = EfficientNet_GCShift(device, num_segments, batch_size).to(device)
opt3 = optim.SGD(EFN_model_3.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

scheduler3 = optim.lr_scheduler.MultiStepLR(opt3, milestones=milestones, gamma=gamma)
# scheduler3 = optim.lr_scheduler.ReduceLROnPlateau(opt3, mode='min', factor=0.3, patience=3)
train3_acc = []
test3_acc = []

torchsummary.summary(EFN_model_3, (num_segments, 3, image_size, image_size))

In [ ]:
train3_acc, test3_acc = train(EFN_model_3, epochs, criterion, train3_dataloader, test3_dataloader, opt3, scheduler3, train3_size, test3_size, device, modelName+"_3.pt")